In [1]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2


In [2]:
print(os.getcwd())

base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

/home/aiden/src/dacon_14


(297571, 3)

### Mecab

In [3]:
# mecab = Mecab()
# # df_space['morphs'] = df_space['spacing'].apply(lambda x: mecab.morphs(x))
# df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))

# df_fea['nouns_str'] = df_fea['nouns'].apply(lambda x: ' '.join(x))

# df_fea.to_pickle('df_fea.pkl')

df_fea = pd.read_pickle('data/df_fea.pkl')

### Count

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_df=0.25, min_df=50)

vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
cnt_vec = vectorizer.transform(df_fea['nouns_str'].values).toarray()

cnt_dict = {'cnt_{0:03d}'.format(i):'cnt_{0:03d}_{1}'.format(i, c) for i, c in enumerate(vectorizer.get_feature_names())}
cnt_cols = sorted(cnt_dict.keys())

df_cnt_vec = pd.DataFrame(cnt_vec, columns=cnt_cols, dtype=np.uint8)
df_cnt_vec.shape

(297571, 860)

### tfidf

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_size = None
stop_words = [
#    '은행',
#     '고객',
#     '가능',
#     '전화',
#     '기간',
#     '대출',
#     '금리',
#     '상담',
#     '광고',
#     '상품',
#     '센터',
]
vectorizer = TfidfVectorizer(max_features=tfidf_size, 
                             stop_words=stop_words, 
                             max_df=0.5,
                             min_df=100)

# vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
vectorizer = vectorizer.fit(df_fea['nouns_str'].values)

tfidf = vectorizer.transform(df_fea['nouns_str'].values).toarray()
tfidf_dict = {'tfidf_{0:03d}'.format(v):'tfidf_{0:03d}_{1}'.format(v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
tfidf_cols = sorted(tfidf_dict.keys())

df_tfidf = pd.DataFrame(tfidf, columns=tfidf_cols, dtype=np.float16)
df_tfidf.shape

(297571, 2900)

In [6]:
vectorizer = TfidfVectorizer(max_features=tfidf_size, 
                             stop_words=stop_words, 
                            # max_df=0.5,min_df=100
                            )

vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
# vectorizer = vectorizer.fit(df_fea['nouns_str'].values)

tfidf = vectorizer.transform(df_fea['nouns_str'].values).toarray()
tfidf2_dict = {'tfidf2_{0:03d}'.format(v):'tfidf2_{0:03d}_{1}'.format(v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
tfidf2_cols = sorted(tfidf2_dict.keys())

df_tfidf2 = pd.DataFrame(tfidf, columns=tfidf2_cols, dtype=np.float16)
df_tfidf2.shape

(297571, 2773)

### Merged

In [7]:
print(df_fea.shape, df_cnt_vec.shape, df_tfidf.shape, df_tfidf2.shape)

df_merged = pd.concat([df_fea.reset_index(), df_cnt_vec[cnt_cols], df_tfidf[tfidf_cols], df_tfidf2[tfidf2_cols]], axis=1)
print(df_merged.shape)
# df_merged = pd.concat([df_merged, df_tfidf[tfidf_cols]], axis=1)
# print(df_merged.shape)

print(df_merged.info())
df_merged.head(10)

(297571, 5) (297571, 860) (297571, 2900) (297571, 2773)
(297571, 6539)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 297571 entries, 0 to 297570
Columns: 6539 entries, id to tfidf2_999
dtypes: float16(5673), int64(2), object(4), uint8(860)
memory usage: 3.4+ GB
None


,id,year_month,text,smishing,nouns,nouns_str,cnt_000,cnt_001,cnt_002,cnt_003,...,tfidf2_990,tfidf2_991,tfidf2_992,tfidf2_993,tfidf2_994,tfidf2_995,tfidf2_996,tfidf2_997,tfidf2_998,tfidf2_999
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[행성, 산, 팀장, 행복, 주말]",행성 산 팀장 행복 주말,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 하루, 시작, 은행, 진월동, 라운지]",오늘 하루 시작 은행 진월동 라운지,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]",안녕 고객 은행 금일 납부 금액 원 감사 새해 복 은행 옥포,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...",고객 안녕 은행 지점 해 동안 저희 지점 성원 감사 시작 년 소망 일 고객 가정 건...,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[월, 한, 행복, 해]",월 한 행복 해,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,"[행복, 주말, 현남, 전담, 직원, 대리]",행복 주말 현남 전담 직원 대리,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 ...,0,"[고객, 안녕, 은행, 무교, 지점, 과장, 아침, 눈, 눈, 세상, 적, 눈, 눈...",고객 안녕 은행 무교 지점 과장 아침 눈 눈 세상 적 눈 눈 순간 출근 걱정 어른 ...,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하...,0,"[고객, 한, 은혜, 진심, 감사, 가슴, 간직, 정유, 년, 새해, 가족, 행복,...",고객 한 은혜 진심 감사 가슴 간직 정유 년 새해 가족 행복 뜻 바 진심 소망 은행...,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,"[설, 연휴, 가족, 정, 추억, 오]",설 연휴 가족 정 추억 오,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1,"[광고, 고객, 뒤, 마음가짐, 준비, 당, 행상, 품, 자격, 기준, 심사, 기준...",광고 고객 뒤 마음가짐 준비 당 행상 품 자격 기준 심사 기준 완화 상품 상품 정보...,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.112427


In [8]:
del df_fea, df_cnt_vec, df_tfidf, df_tfidf2

In [9]:
train_size = len(df_train)
print(train_size)

cat_cols = []
fea_cols = cnt_cols + tfidf_cols + cat_cols + tfidf2_cols
fea_cols = tfidf_cols + cat_cols + tfidf2_cols
# fea_cols = cnt_cols + cat_cols
# fea_cols = tfidf_cols + cat_cols
len(fea_cols)

295945


5673

### Model

In [10]:
def eval_summary(y_true, y_score, cut_off=0.5):
    y_pred = y_score.copy()
    y_pred[y_pred > cut_off] = 1
    y_pred[y_pred <= cut_off] = 0

    eval_dict = {}
    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_score, pos_label=1)
    
    eval_dict['auc'] = metrics.auc(fpr, tpr)
    eval_dict['confusion_matrix'] = metrics.confusion_matrix(y_true, y_pred)
    
    pre, rec, _, _ = metrics.precision_recall_fscore_support(y_true, y_pred, pos_label=1)
    eval_dict['precision'] = pre[1]
    eval_dict['recall'] = rec[1]
    
    return eval_dict

#### torch RNN

In [11]:
import torch
print(torch.__version__)
import torch.nn.functional as F

print(torch.cuda.get_device_name(0))
print(torch.cuda.is_available())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

1.3.1
GeForce RTX 2070 SUPER
True


device(type='cuda', index=0)

In [ ]:
class RNN(nn.Module):
    def __init__(self,input_size,embed_size,hidden_size,output_size,num_layers=1,bidirec=False):
        super(RNN,self).__init__()
        
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        if bidirec:
            self.num_directions = 2
        else:
            self.num_directions = 1
            
        self.embed = nn.Embedding(input_size,embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size,num_layers,batch_first=True,bidirectional=bidirec)
        self.linear = nn.Linear(hidden_size*self.num_directions,output_size)
        
    def init_hidden(self,batch_size):
        # (num_layers * num_directions, batch_size, hidden_size)
        hidden = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        cell = Variable(torch.zeros(self.num_layers*self.num_directions,batch_size,self.hidden_size))
        return hidden, cell
    
    def forward(self,inputs):
        """
        inputs : B,T
        """
        embed = self.embed(inputs) # word vector indexing
        hidden, cell = self.init_hidden(inputs.size(0)) # initial hidden,cell
        
        output, (hidden,cell) = self.lstm(embed,(hidden,cell))
        
        # Many-to-Many
        output = self.linear(output) # B,T,H -> B,T,V
        
        # Many-to-One
        #hidden = hidden[-self.num_directions:] # (num_directions,B,H)
        #hidden = torch.cat([h for h in hidden],1)
        #output = self.linear(hidden) # last hidden
        
        return output

In [ ]:
# del model, x_data, y_data

In [ ]:
torch.cuda.init()
torch.cuda.empty_cache() 

In [ ]:
model_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
x_data = torch.Tensor(df_merged[:train_size][fea_cols].values).to(device)
y_data = torch.Tensor(df_merged[:train_size][['smishing']].values).to(device)

model = MyModel()
model.to(device)

criterion = torch.nn.BCELoss(size_average=True)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

epoch = 0

In [ ]:
for e in range(epoch, epoch + 500000):
    y_pred = model(x_data)
    
    loss = criterion(y_pred, y_data)

    if e % 100 == 0:
        print(e, loss.item())
        
    if e % 10000 == 0:
        y_score = model(x_data).cpu().detach().numpy().reshape(1, -1)[0]
        print(eval_summary(df_merged[:train_size]['smishing'].values, y_score, cut_off=0.5))
        
        x_test = torch.Tensor(df_merged[train_size:][fea_cols].values).to(device)
        df_test['smishing_{}'.format(e)] = model(x_test).cpu().detach().numpy().reshape(1, -1)[0]
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    epoch = e
    

In [ ]:
y_score = model(x_data).cpu().detach().numpy().reshape(1, -1)[0]
eval_summary(df_merged[:train_size]['smishing'].values, y_score, cut_off=0.5)

In [ ]:
x_test = torch.Tensor(df_merged[train_size:][fea_cols].values).to(device)
pred = model(x_test).cpu().detach().numpy().reshape(1, -1)[0]

In [ ]:
model_ts

In [ ]:
df_test['smishing'] = pred
df_test['smishing'].hist(bins=100, figsize=(20, 5))

In [ ]:
torch.save(model.state_dict(), 'model/torch_lr.model')

In [ ]:
df_test[['id', 'smishing']].to_csv('torch_lr_{}.csv'.format(lgb_ts), index=False)
df_test[['id', 'smishing', 'text']].sort_values('smishing', ascending=False).to_csv('torch_lr_{}_text.csv'.format(model_ts), index=False)